In [1]:
import os
import pyaudio
import wave
from tensorflow import keras
import  numpy as np
import time

In [29]:
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range (0,numdevices):
        if p.get_device_info_by_host_api_device_index(0,i).get('maxInputChannels')>0:
                print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name'))

        if p.get_device_info_by_host_api_device_index(0,i).get('maxOutputChannels')>0:
                print("Output Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name'))


Output Device id  0  -  HDA NVidia: HDMI 0 (hw:0,3)
Output Device id  1  -  HDA NVidia: HDMI 1 (hw:0,7)
Output Device id  2  -  HDA NVidia: HDMI 2 (hw:0,8)
Output Device id  3  -  HDA NVidia: HDMI 3 (hw:0,9)
Input Device id  4  -  USB PnP Audio Device: Audio (hw:1,0)
Input Device id  5  -  HD-Audio Generic: ALC892 Analog (hw:2,0)
Output Device id  5  -  HD-Audio Generic: ALC892 Analog (hw:2,0)
Input Device id  6  -  HD-Audio Generic: ALC892 Alt Analog (hw:2,2)
Input Device id  7  -  webcam: USB Audio (hw:3,0)
Output Device id  8  -  hdmi
Input Device id  9  -  pulse
Output Device id  9  -  pulse
Input Device id  10  -  default
Output Device id  10  -  default


In [ ]:
files = os.listdir("/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/")
for i,f in enumerate(files):
    files[i] = "/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/"+f

In [5]:
from pydub import AudioSegment

audio = AudioSegment.from_file(files[0])
audio2 = AudioSegment.from_file("/home/andrea/Scrivania/rec.wav")
s1 = audio.get_array_of_samples()
s2 = audio2.get_array_of_samples()

MODEL TEST

In [2]:

model = keras.models.load_model("audio_model_bidirectional_LSTM.h5")

dataset = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/dataset.npy")
std_dev = np.std(dataset)
mean = np.mean(dataset)
maximum = np.max(dataset)
min = np.min(dataset)
scale = max(maximum, abs(min))

def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [34]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 48000

In [45]:
frames = []
do_prediction = False
window = np.zeros((1,16,16516))
filled = 0
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_device_index=4,
                frames_per_buffer=CHUNK)
add = False
start = time.time()

while True:
    st1 = time.time()
    data = stream.read(CHUNK,exception_on_overflow = False)
    frames.append(data)
    #print("Read stream in :" + str(time.time()-st1))
    if CHUNK*len(frames) == 16384:

        s_arr = np.zeros((1,1,16516))
        s_arr[0,0,:16384] = np.frombuffer(b''.join(frames), dtype=np.int16)
        frames = []
        add = True
        end = time.time()
        #print("Total time to fill a frame: " + str(end-start))
        start = end
        #print("Full frame")
    if filled <= 15 and add:
        window[0,filled,:] = normalize(s_arr[0,0,:], std_dev, mean, scale)
        filled += 1

    elif filled > 15 and add:
        window = np.hstack((np.delete(window, 0, 1), normalize(s_arr, std_dev, mean, scale)))
        print("Full Window")
        do_prediction = True
    add = False
    if do_prediction:
        c = model.predict_classes(window)
        #do_prediction = False
        print(c)

Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]

KeyboardInterrupt: 

In [44]:
stream.stop_stream()
stream.close()
p.terminate()

In [47]:
t_base = np.zeros((1,15,64))
t_add = np.ones((1,1,64,))

In [48]:
merged = np.hstack((t_base,t_add))

In [9]:
import chardet
the_encoding = chardet.detect(frames[0])['encoding']

In [22]:
decoded =  np.fromstring(b''.join(frames), 'Float32')
decoded2 = np.frombuffer(frames[0], dtype=np.int16)

/home/andrea/Scrivania/RC_Carta/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  """Entry point for launching an IPython kernel.
/home/andrea/Scrivania/RC_Carta/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """Entry point for launching an IPython kernel.
